In [1]:
# get api key from your .env file
import os
from dotenv import load_dotenv

load_dotenv()
API_KEY = os.getenv('Nasdaq_API_key')

#print(API_KEY)

In [2]:
# requests module
import requests
import json
import pandas as pd

In [3]:
# Now, call the Nasdaq API and pull out a small sample of the data (only one day) to get a glimpse
# into the JSON structure that will be returned
url = 'https://data.nasdaq.com/api/v3/datasets/FSE/AFX_X/data.json?'

DataSet = 'FSE'
Ticker = 'AFX_X'

querystring_1 = {'database_code':DataSet,
               'dataset_code' :Ticker,
               'start_date' : '2017-01-01',
               'end_date' :   '2017-01-02',
               'api_key': API_KEY}

response_1 = requests.get(url,params = querystring_1) 

In [4]:
# Inspecting Nasdaq API Jason format

print(json.dumps(response_1.json(), indent =4))

{
    "dataset_data": {
        "limit": null,
        "transform": null,
        "column_index": null,
        "column_names": [
            "Date",
            "Open",
            "High",
            "Low",
            "Close",
            "Change",
            "Traded Volume",
            "Turnover",
            "Last Price of the Day",
            "Daily Traded Units",
            "Daily Turnover"
        ],
        "start_date": "2017-01-01",
        "end_date": "2017-01-02",
        "frequency": "daily",
        "data": [
            [
                "2017-01-02",
                34.99,
                35.94,
                34.99,
                35.8,
                null,
                44700.0,
                1590561.0,
                null,
                null,
                null
            ]
        ],
        "collapse": null,
        "order": null
    }
}


In [5]:
# Data for the year 2017

querystring = {'database_code':DataSet,
               'dataset_code' :Ticker,
               'start_date' : '2017-01-01',
               'end_date' :   '2017-12-31',
               'api_key': API_KEY}

response_2017 = requests.get(url,params = querystring) 

In [6]:
json_data = response_2017.json()

type(json_data) # the response.json() is a python dictionary

dict

Transform dictionary into data frame for easy of data use and manipulation

In [7]:
 n = len(json_data['dataset_data']['data']) # length of data

In [8]:
df = pd.DataFrame.from_dict(json_data['dataset_data']['data']) # Transfor dictionary to pands data frame

df.columns = json_data['dataset_data']['column_names']

In [9]:
df.head()

,Date,Open,High,Low,Close,Change,Traded Volume,Turnover,Last Price of the Day,Daily Traded Units,Daily Turnover
0,2017-12-29,51.76,51.94,51.45,51.76,NaN,34640.0,1792304.0,None,None,None
1,2017-12-28,51.65,51.82,51.43,51.60,NaN,40660.0,2099024.0,None,None,None
2,2017-12-27,51.45,51.89,50.76,51.82,NaN,57452.0,2957018.0,None,None,None
3,2017-12-22,51.05,51.50,50.92,51.32,NaN,71165.0,3641949.0,None,None,None
4,2017-12-21,51.16,51.52,50.90,51.40,NaN,120649.0,6179433.0,None,None,None


In [10]:
# drop columns we wont use
cols = ['Change','Turnover','Last Price of the Day','Daily Traded Units','Daily Turnover']

df.drop(columns = cols, inplace = True)


In [11]:
# Calculate what the highest and lowest opening prices were for the stock in this period.

print(df.Open.max(),df.Open.min()) # Highest open was 53.11 and Lowest was 34.00

53.11 34.0


In [12]:
# What was the largest change in any one day (based on High and Low price)?

df['Range'] = df.High - df.Low

df.Range.max() # Maximum range (difference between high and low) was 2.81

2.8100000000000023

In [13]:
# What was the largest change between any two days (based on Closing Price)?

df['Closing Diff'] = df.Close.diff()

df['Closing Diff'].max() # Maximum price difference between two days was 2.56

2.559999999999995

In [14]:
# What was the average daily trading volume during this year?

df['Traded Volume'].mean() # Average daily trading volume was 89,124 trades

89124.33725490196

In [15]:
#(Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

n = len(df) # there are an odd number of trading days 

df['Traded Volume'].sort_values().reset_index(drop = True)[(n+1)/2] # median trading volume was 76,600

76600.0